# Demo 04 - Duplicates and Cardinality

In the first notebook, we saw some fishy behavior with respect to increases in expenses in the year 2018 versus prior years.  Now let's use a couple tools at our disposal to delve further into the problem.

In [ ]:
if(!require(tidyverse)) {
    install.packages("tidyverse", repos = "http://cran.us.r-project.org")
    library(tidyverse)
}

if(!require(odbc)) {
    install.packages("odbc", repos = "http://cran.us.r-project.org")
    library(odbc)
}

if(!require(data.table)) {
  install.packages("data.table", repos = "http://cran.us.r-project.org")
  library(data.table)
}

## Count of Invoices by Day

The first thing we want to do is see how many invoices we get per bus, vendor, and day.  Our organization requires vendors send one invoice per bus maintenance item, so we generally expect no more than one invoice per combination of bus, vendor, and day.  After all, invoices can get lost in the shuffle and creating more than one is inefficient.

In [ ]:
conn <- DBI::dbConnect(odbc::odbc(), 
                      Driver = "SQL Server", 
                      Server = "localhost", 
                      Database = "ForensicAccounting", 
                      Trusted_Connection = "True")

In [ ]:
dupes <- DBI::dbGetQuery(conn, "WITH records AS
(
	SELECT
		li.LineItemDate,
		li.BusID,
		li.VendorID,
		COUNT(*) AS NumberOfInvoices
	FROM dbo.LineItem li
	GROUP BY
		li.LineItemDate,
		li.BusID,
		li.VendorID
)
SELECT
	NumberOfInvoices,
	COUNT(*) AS NumberOfOccurrences
FROM records
GROUP BY
	NumberOfInvoices
ORDER BY
	NumberOfInvoices;")

In [ ]:
dupes

It looks like for the vast majority of the time, we see one invoice per bus, vendor, and day.  569 times we have 2 invoices and we got 26 invoices on a single day once.  It might be interesting to see who's sending us multiple invoices so let's do that.

In [ ]:
dupeSenders <- DBI::dbGetQuery(conn, "WITH records AS
(
	SELECT
		li.LineItemDate,
		li.BusID,
		li.VendorID,
		COUNT(*) AS NumberOfInvoices
	FROM dbo.LineItem li
	GROUP BY
		li.LineItemDate,
		li.BusID,
		li.VendorID
)
SELECT
	VendorID,
	COUNT(*) AS NumberOfOccurrences
FROM records
WHERE
	NumberOfInvoices > 1
GROUP BY
	VendorID
ORDER BY
	VendorID;")

In [ ]:
dupeSenders

Vendors 2, 5, and 9 have double-digit counts of double-invoice days, but vendor 5 has four times as many days as the next-highest.  That's a little weird and worth keeping in the back of our minds, but it's not outlandish.  So let's keeep digging.

## Cardinality Checks

We can use the `rapply()` function to perform cardinality checks, showing us how many distinct values there are in our data set.

In [ ]:
lineItems <- DBI::dbGetQuery(conn, "SELECT
	*
FROM dbo.LineItem li;")

In [ ]:
rapply(lineItems, function(x) { length(unique(x)) })

We knew that there were 15 vendors, 28 expense categories, and 12 employees, so those aren't surprising.  We do see 460 buses which have had maintenance of some sort done on them.  That means 36 buses were retired without ever having gone through maintenance.

Cardinality is also useful when looking at subsets of data.  For example, let's filter to include just invoices valued between \\$850 and \\$999.99, as these are high-value invoices which fall below the two-signer rule.

In [ ]:
highValueInvoices <- lineItems %>% dplyr::filter(Amount >= 850 & Amount < 1000)

In [ ]:
rapply(highValueInvoices, function(x) { length(unique(x)) })

It looks like 12 of our 15 vendors invoices between \\$850 and \\$999.99.  We can dig deeper using the `setDT()` function in the `data.table` package.  Let's look at counts by vendor ID:

In [ ]:
data.table::setDT(highValueInvoices)[, .N, keyby=VendorID]

Vendor 5 seems to keep coming up as an outlier.  Maybe they are special but if your spidey-senses are tingling, I don't blame you.

Maybe they just have a lot of high-value items, so let's see how many invoices over \$1000 they have in our data set.

In [ ]:
data.table::setDT(filter(lineItems, Amount > 1000))[, .N, keyby=VendorID]

That's strange.  They have no invoices over \\$1000.  How about ones which are just under \\$1000?  Here we will focus specifically on vendor 5 and look at amounts greater than \\$995.

In [ ]:
data.table::setDT(filter(lineItems, VendorID == 5 & Amount > 995))[, .N, keyby=Amount]

1468 invoices all happen to be within \$5 of our two-signer cutoff?  And 1446 of those happen to be one penny short?  The circumstantial evidence is starting to add up.  We don't have anything yet but this is looking very suspicious.

Now let's pivot and look at high-value invoices by employee.

In [ ]:
data.table::setDT(highValueInvoices)[, .N, keyby=EmployeeID]

All twelve of our employees have dealt with high-value invoices.  Let's see what it looks like when we filter on the suspicious vendor.

In [ ]:
data.table::setDT(filter(lineItems, VendorID == 5 & Amount > 995))[, .N, keyby=EmployeeID]

Only four employees handled those invoices for vendor 5.  But maybe the agency has people focus on certain sets of vendors.  Let's limit ourselves to the year 2018 and see how many invoices for vendor 5 each employee has handled.

In [ ]:
data.table::setDT(filter(lineItems, VendorID == 5 & year(LineItemDate) == 2018))[, .N, keyby=EmployeeID]

All 12 have handled invoices.  8 of the 12 have taken a couple dozen but our final four have over 2000 apiece.  That's averaging 6-7 invoices per employee per day just for that vendor.  That's a lot of invoices!